In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mipt-bio-2025/fashion-mnist-testn/fmnist_test.csv
/kaggle/input/mipt-bio-2025/fashion-mnist-testn/sample_submission.csv
/kaggle/input/mipt-bio-2025/fashion-mnist-testn/fmnist_train.csv
/kaggle/input/test-fashion-mnist/fmnist_test.csv
/kaggle/input/test-fashion-mnist/sample_submission.csv
/kaggle/input/test-fashion-mnist/fmnist_train.csv


In [2]:
# Загрузка библиотек
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np


In [3]:
# Проверяем GPU, если его нет - используем CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


In [4]:
# Загружаем тренировочные и тестовые данные из csv файлов
train_df = pd.read_csv('/kaggle/input/test-fashion-mnist/fmnist_train.csv')
test_df = pd.read_csv('/kaggle/input/test-fashion-mnist/fmnist_test.csv')
# Удаляем пропуски
train_df = train_df.dropna()

In [5]:
# Класс Dataset для удобной работы с данными в PyTorch
class FashionMNISTDataset(Dataset):
    def __init__(self, df, train=True):
        self.train = train
        if self.train:
            self.labels = torch.tensor(df['label'].values, dtype=torch.long)
            self.features = torch.tensor(df.drop(['label', 'Id'], axis=1).values, dtype=torch.float32).view(-1, 1, 28, 28) / 255.0
        else:
            self.features = torch.tensor(df.drop('Id', axis=1).values, dtype=torch.float32).view(-1, 1, 28, 28) / 255.0
            self.ids = df['Id'].values

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        if self.train:
            return self.features[idx], self.labels[idx]
        return self.features[idx], self.ids[idx]


In [6]:
# Создаем объекты Dataset и DataLoader для загрузки батчами
train_dataset = FashionMNISTDataset(train_df)
test_dataset = FashionMNISTDataset(test_df, train=False)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [7]:
# Создаем базовую архитектуру сверточной нейронной сети
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            # Первый сверточный блок: Conv2d -> BatchNorm -> ReLU -> Conv2d -> BatchNorm -> ReLU -> MaxPool -> Dropout
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),

            # Второй сверточный блок
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),

            # Плоское представление и полносвязные слои с BatchNorm, ReLU и Dropout
            nn.Flatten(),
            nn.Linear(64*7*7, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)  # 10 классов для FashionMNIST
        )

    def forward(self, x):
        return self.model(x)


In [8]:
# Класс ансамбля, объединяющего несколько моделей CNN
class EnsembleModel(nn.Module):
    def __init__(self, n_models=3):
        super().__init__()
        # Список из n_models экземпляров CNNModel
        self.models = nn.ModuleList([CNNModel() for _ in range(n_models)])

    def forward(self, x):
        # Получаем предсказания от каждой модели
        outputs = [model(x) for model in self.models]
        # Усредняем выходы для итогового предсказания ансамбля
        avg_output = torch.mean(torch.stack(outputs), dim=0)
        return avg_output



In [9]:
# Функция для вычисления accuracy
def accuracy(preds, labels):
    return (preds == labels).float().mean().item()

In [10]:
# Гиперпараметры обучения
EPOCHS = 100
LR = 0.001
N_ENSEMBLE = 3

In [11]:
# Инициализируем ансамбль и оптимизатор
ensemble_model = EnsembleModel(n_models=N_ENSEMBLE).to(device)
optimizer = optim.Adam(ensemble_model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

In [12]:
# Цикл обучения с выводом loss и accuracy по эпохам
for epoch in range(EPOCHS):
    ensemble_model.train()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = ensemble_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        preds = torch.argmax(outputs, dim=1)
        running_corrects += (preds == labels).sum().item()
        total_samples += inputs.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects / total_samples
    print(f'Epoch {epoch+1}/{EPOCHS} Loss: {epoch_loss:.4f} Accuracy: {epoch_acc:.4f}')

Epoch 1/100 Loss: 0.4796 Accuracy: 0.8276
Epoch 2/100 Loss: 0.3087 Accuracy: 0.8870
Epoch 3/100 Loss: 0.2605 Accuracy: 0.9043
Epoch 4/100 Loss: 0.2247 Accuracy: 0.9182
Epoch 5/100 Loss: 0.2012 Accuracy: 0.9275
Epoch 6/100 Loss: 0.1831 Accuracy: 0.9337
Epoch 7/100 Loss: 0.1700 Accuracy: 0.9381
Epoch 8/100 Loss: 0.1493 Accuracy: 0.9469
Epoch 9/100 Loss: 0.1348 Accuracy: 0.9502
Epoch 10/100 Loss: 0.1258 Accuracy: 0.9538
Epoch 11/100 Loss: 0.1199 Accuracy: 0.9562
Epoch 12/100 Loss: 0.0961 Accuracy: 0.9665
Epoch 13/100 Loss: 0.0966 Accuracy: 0.9655
Epoch 14/100 Loss: 0.0975 Accuracy: 0.9653
Epoch 15/100 Loss: 0.0827 Accuracy: 0.9708
Epoch 16/100 Loss: 0.0734 Accuracy: 0.9748
Epoch 17/100 Loss: 0.0621 Accuracy: 0.9780
Epoch 18/100 Loss: 0.0778 Accuracy: 0.9720
Epoch 19/100 Loss: 0.0599 Accuracy: 0.9787
Epoch 20/100 Loss: 0.0548 Accuracy: 0.9804
Epoch 21/100 Loss: 0.0472 Accuracy: 0.9831
Epoch 22/100 Loss: 0.0480 Accuracy: 0.9832
Epoch 23/100 Loss: 0.0423 Accuracy: 0.9856
Epoch 24/100 Loss: 0

In [13]:
# Переключаем модель в режим оценки для предсказаний на тесте
ensemble_model.eval()
predictions = []
ids_list = []

with torch.no_grad():
    for inputs, ids in test_loader:
        inputs = inputs.to(device)
        outputs = ensemble_model(inputs)
        preds = torch.argmax(outputs, dim=1)
        predictions.extend(preds.cpu().numpy())
        ids_list.extend(ids.numpy())




In [14]:
# Формируем DataFrame для submission и сохраняем в csv
submission = pd.DataFrame({'Id': ids_list, 'label': predictions})
submission.to_csv('submission_ensemble.csv', index=False)
print('Файл submission_ensemble.csv успешно создан!')

Файл submission_ensemble.csv успешно создан!
